In [1]:
import numpy as np
import pandas as pd

### Define Functions

In [2]:
def clean_fips(dataframe):
    new_list = []
    for i in range(len(dataframe)):
        if len(str(dataframe['id'][i]))==4:
            new_list.append(str('0')+str(dataframe['id'][i]))
        elif len(str(dataframe['id'][i]))==5:
            new_list.append(str(dataframe['id'][i]))
        else:
            new_list.append(np.nan)
    dataframe['id'] = new_list
    return dataframe

### Independent Variables

In [3]:
# Open Files
## County Health Rankings
rank_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/County_Health_Ranks/County_Healthranks.csv')

## Educational Attainment
eduattain_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/Educational_Attainment/eduattain.csv')

## English Speaking
english_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/English/english.csv')

## Poverty
poverty_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/Poverty/poverty.csv')

## Race and Ethnicity
race_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/Race/race.csv')

## House Price Index
hpi_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/HousePrice/County_HPI.csv')

## Median Income
income_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/Income/income.csv')

## Population
pop_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/Population/population.csv', encoding='latin1')

# Clean Files
rank_df = clean_fips(rank_df)
eduattain_df = clean_fips(eduattain_df)
english_df = clean_fips(english_df)
poverty_df = clean_fips(poverty_df)
race_df = clean_fips(race_df)
hpi_df = clean_fips(hpi_df)
income_df = clean_fips(income_df)
pop_df = clean_fips(pop_df)

- Time invariant

In [4]:
# Metropolitan, non-metropolitan

# Rural, non-rural
nchs_df = pd.read_excel(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/Census_NCHS_UrbanRural.xlsx')
# Land area
land_df = pd.read_excel(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Census/County_land_area_2010.xlsx')


# Clean Files
nchs_df = clean_fips(nchs_df)
land_df = clean_fips(land_df)


/usr/local/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Dependent Variables

In [5]:
dv_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/iv/Market_Saturation_And_Utilization_Dataset_2021-04-16.csv')
dv_df = dv_df[dv_df['Type_of_Service']=='Telemedicine']
dv_df = dv_df[dv_df['Aggregation_Level']=='COUNTY'] # County
dv_df = dv_df[dv_df['Reference_Period'].isin(['2019-01-01 to 2019-12-31',
                                              '2018-01-01 to 2018-12-31',
                                              '2017-01-01 to 2017-12-31'])] # Reference dates
dv_df = dv_df.reset_index(drop=True)

In [6]:
## Clean FIPS
## State
new_list = []
for i in range(len(dv_df)):
    if len(str(int(dv_df['State_FIPS'][i])))==1:
        new_list.append(str('0')+str(int(dv_df['State_FIPS'][i])))
    elif len(str(int(dv_df['State_FIPS'][i])))==2:
        new_list.append(str(int(dv_df['State_FIPS'][i])))
    else:
        print(i)
dv_df['State_FIPS'] = new_list

## County
new_list = []
for i in range(len(dv_df)):
    if len(str(int(dv_df['County_FIPS'][i])))==1:
        new_list.append(str('00')+str(int(dv_df['County_FIPS'][i])))
    elif len(str(int(dv_df['County_FIPS'][i])))==2:
        new_list.append(str('0')+str(int(dv_df['County_FIPS'][i])))
    elif len(str(int(dv_df['County_FIPS'][i])))==3:
        new_list.append(str(int(dv_df['County_FIPS'][i])))
    else:
        print(i)
dv_df['County_FIPS'] = new_list

dv_df['id'] = [str(dv_df['State_FIPS'][i])+str(dv_df['County_FIPS'][i]) for i in range(len(dv_df))]

dv_df['Area Name'] = [dv_df['County'][i]+", "+dv_df['State'][i] for i in range(len(dv_df))]

# Filter columns
dv_df = dv_df[['Reference_Period', 'id', 'Area Name', 
               'Number_of_Users', 'Percentage_of_Users_out_of_FFS_Beneficiaries','Number_of_Fee-for-Service_Beneficiaries', 
               'Number_of_Providers', 
               'Total_Payment','State', 'County', 'State_FIPS', 'County_FIPS']]

In [10]:
dv_df['Reference_Period'] = [str(i)[:4] for i in dv_df['Reference_Period']]
dv_df = dv_df.rename(columns={'Reference_Period':'year'})

In [14]:
dv_df

,year,id,Area Name,Number_of_Users,Percentage_of_Users_out_of_FFS_Beneficiaries,Number_of_Fee-for-Service_Beneficiaries,Number_of_Providers,Total_Payment,State,County,State_FIPS,County_FIPS
0,2017,01003,"Baldwin, AL",84,0.28,30148,1,15576.83,AL,Baldwin,01,003
1,2017,01005,"Barbour, AL",94,2.01,4669,4,10075.74,AL,Barbour,01,005
2,2017,01009,"Blount, AL",37,0.57,6520,1,3021.95,AL,Blount,01,009
3,2017,01011,"Bullock, AL",33,2.43,1356,1,2408.88,AL,Bullock,01,011
4,2017,01013,"Butler, AL",61,1.54,3952,2,6931.28,AL,Butler,01,013
...,...,...,...,...,...,...,...,...,...,...,...,...
6645,2019,56039,"Teton, WY",11,0.31,3512,0,730.31,WY,Teton,56,039
6646,2019,56041,"Uinta, WY",24,0.73,3299,1,4769.48,WY,Uinta,56,041
6647,2019,56043,"Washakie, WY",13,0.65,1991,0,1259.78,WY,Washakie,56,043
6648,2019,56045,"Weston, WY",22,1.37,1606,1,2885.81,WY,Weston,56,045


### Merge data

In [387]:
dv_df['year'] = dv_df['year'].astype(int)
df = pd.merge(rank_df, dv_df.drop(columns=['County','State']), on=['id','year'], how='outer')
df = pd.merge(eduattain_df.drop(columns=['Area Name']), df, on=['id','year'], how='outer')
df = pd.merge(english_df.drop(columns=['Area Name']), df, on=['id','year'], how='outer')
df = pd.merge(poverty_df.drop(columns=['Area Name', 'State']), df, on=['id','year'], how='outer')
df = pd.merge(race_df.drop(columns=['Area Name']), df, on=['id','year'], how='outer')
df = pd.merge(hpi_df.drop(columns=['County','State']), df, on=['id','year'], how='outer')
df = pd.merge(income_df.drop(columns=['Area Name']), df, on=['id','year'], how='outer')
df = pd.merge(nchs_df.drop(columns=['County','State']), df, on=['id'], how='outer')
df = pd.merge(land_df.drop(columns=['Area Name']), df, on=['id'], how='outer')
df = pd.merge(pop_df.drop(columns=['State','County']), df, on=['id'], how='outer')

# Sort
df = df.dropna(subset=['id'])
df = df.sort_values(by=['id','year'])


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9520 entries, 0 to 9519
Data columns (total 67 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   id                                                      9520 non-null   object 
 1   POPESTIMATE2017                                         9477 non-null   float64
 2   POPESTIMATE2018                                         9477 non-null   float64
 3   POPESTIMATE2019                                         9477 non-null   float64
 4   total_land_water                                        9477 non-null   float64
 5   land_area                                               9477 non-null   float64
 6   water_area                                              9477 non-null   float64
 7   code                                                    9433 non-null   float64
 8   nchs_rurality                         

##### Population Density

In [ ]:
density_df = df[['id','POPESTIMATE2017','POPESTIMATE2018','POPESTIMATE2019','total_land_water', 'land_area']].dropna(subset=['POPESTIMATE2017','land_area'])
density_df = density_df.drop_duplicates()

density_df['density_2017'] = density_df['POPESTIMATE2017']/density_df['land_area']
density_df['density_2018'] = density_df['POPESTIMATE2018']/density_df['land_area']
density_df['density_2019'] = density_df['POPESTIMATE2019']/density_df['land_area']

In [51]:
df = pd.merge(df, density_df.drop(columns=['total_land_water','land_area',
                                           'POPESTIMATE2017','POPESTIMATE2018','POPESTIMATE2019']), on=['id'], how='outer')

##### Hospital Density

- First we need to make a 'id' column for the hospital database

In [52]:
fips_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/FIPS/fips_df2.csv')
fips_df = fips_df.rename(columns={'State_id':'State'})

In [53]:
hosp_df = pd.read_excel(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/HIMSS/dbo_HAEntity_2017_county.xlsx')
hosp_df = hosp_df[hosp_df['HAEntityType']=='Hospital']
hosp_df = hosp_df.sort_values(by=['State','County'])

In [54]:
# # Clean county names
hosp_df['county_clean'] = [i.lower() for i in hosp_df['County']]
hosp_df['county_clean'] = [i.replace(' ','') for i in hosp_df['county_clean']]
hosp_df['county_clean'] = hosp_df['county_clean'].replace(to_replace={"'":'',
                                                                      '-':'',
                                                                      '(borough)':'',
                                                                      '(censusarea)':'',
                                                                      '(city$)':'',
                                                                      '(cityand)':'',
                                                                      '(cityandborough)':'',
                                                                      '(county)':'',
                                                                      '(municipality)':'',
                                                                      '(municipio)':'',
                                                                      '(parish)':'',
                                                                      '(peninsula)':'',
                                                                      '(st.)':'saint'}, regex=True)
hosp_county_df = hosp_df[['State','county_clean']].drop_duplicates()
hosp_county_df = hosp_county_df.reset_index(drop=True)
hosp_county_df['state_county'] = [str(hosp_county_df['State'][i])+"_"+str(hosp_county_df['county_clean'][i]) for i in range(len(hosp_county_df))]

In [55]:
# Clean county names
fips_df['county_clean'] = [i.lower() for i in fips_df['Area_name']]
fips_df['county_clean'] = [i.replace(' ','') for i in fips_df['county_clean']]
fips_df['county_clean'] = fips_df['county_clean'].replace(to_replace={"'":'',
                                                                      '-':'',
                                                                      '(borough)':'',
                                                                      '(censusarea)':'',
                                                                      '(city$)':'',
                                                                      '(cityand)':'',
                                                                      '(cityandborough)':'',
                                                                      '(county)':'',
                                                                      '(municipality)':'',
                                                                      '(municipio)':'',
                                                                      '(parish)':'',
                                                                      '(peninsula)':'',
                                                                      '(st.)':'saint'}, regex=True)
fips_df['state_county'] = [str(fips_df['State'][i])+"_"+str(fips_df['county_clean'][i]) for i in range(len(fips_df))]
fips_df = fips_df.rename(columns={'FIPS':'id'})
fips_df = clean_fips(fips_df)

In [ ]:
hosp_df = hosp_df.reset_index(drop=True)
hosp_df['state_county'] = [str(hosp_df['State'][i])+"_"+str(hosp_df['county_clean'][i]) for i in range(len(hosp_df))]

# Merge data
hosp_county_df = pd.merge(hosp_county_df, fips_df.drop(columns=['State','county_clean']), on=['state_county'], how='outer').dropna(subset=['State'])
hosp_df = pd.merge(hosp_df, hosp_county_df.drop(columns=['State','county_clean']), on=['state_county'], how='outer')

# Make data frame for id & number of hospitals
hospital_density_df = pd.merge(hosp_df.groupby(['id']).count().reset_index(level=[0])[['id','HAEntityId']], hosp_df[['id','state_county']].drop_duplicates(), on=['id'], how='outer')
hospital_density_df.columns = ['id','numhosp','state_county']
df = pd.merge(df, hospital_density_df, on=['id'], how='outer')
df['hospital_density'] = df['numhosp']/df['land_area']

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9523 entries, 0 to 9522
Data columns (total 73 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   id                                                      9523 non-null   object 
 1   POPESTIMATE2017                                         9477 non-null   float64
 2   POPESTIMATE2018                                         9477 non-null   float64
 3   POPESTIMATE2019                                         9477 non-null   float64
 4   total_land_water                                        9477 non-null   float64
 5   land_area                                               9477 non-null   float64
 6   water_area                                              9477 non-null   float64
 7   code                                                    9433 non-null   float64
 8   nchs_rurality                         

### 1. Population Density

In [142]:
# Filter columns
temp_df = df[['id', 'State','state_county', 'year', 
              'Number_of_Users', 'Percentage_of_Users_out_of_FFS_Beneficiaries',
              'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
              'Total_Payment','density_2017','density_2018', 'density_2019']]

In [151]:
# Exclude alaska and hawaii
temp_df['State_FIPS'] = [str(i)[:2] for i in temp_df['id']]
temp_df = temp_df[~temp_df['State_FIPS'].isin(['02','15','24'])]

<ipython-input-151-48f6cf3b41f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['State_FIPS'] = [str(i)[:2] for i in temp_df['id']]


In [152]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9342 entries, 0 to 9522
Data columns (total 13 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            9342 non-null   object 
 1   State                                         7210 non-null   object 
 2   state_county                                  7296 non-null   object 
 3   year                                          9286 non-null   float64
 4   Number_of_Users                               6520 non-null   float64
 5   Percentage_of_Users_out_of_FFS_Beneficiaries  6520 non-null   float64
 6   Number_of_Fee-for-Service_Beneficiaries       6520 non-null   float64
 7   Number_of_Providers                           6520 non-null   float64
 8   Total_Payment                                 6520 non-null   float64
 9   density_2017                                  9297 non-null   f

In [153]:
new_df = pd.DataFrame()

In [154]:
temp_2017_df = temp_df[temp_df['year']==2017][['id', 'state_county', 'Number_of_Users', 
                                'Percentage_of_Users_out_of_FFS_Beneficiaries',
                                'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
                                'Total_Payment','density_2017']]
temp_2017_df = temp_2017_df.rename(columns={'density_2017':'density'})
temp_2017_df = temp_2017_df.dropna(subset=['density'])
new_df = new_df.append(temp_2017_df)

In [155]:
temp_2018_df = temp_df[temp_df['year']==2018][['id', 'state_county', 'Number_of_Users', 
                                'Percentage_of_Users_out_of_FFS_Beneficiaries',
                                'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
                                'Total_Payment','density_2018']]
temp_2018_df = temp_2018_df.rename(columns={'density_2018':'density'})
temp_2018_df = temp_2018_df.dropna(subset=['density'])
new_df = new_df.append(temp_2018_df)

In [156]:
temp_2019_df = temp_df[temp_df['year']==2019][['id', 'state_county', 'Number_of_Users', 
                                'Percentage_of_Users_out_of_FFS_Beneficiaries',
                                'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
                                'Total_Payment','density_2019']]
temp_2019_df = temp_2019_df.rename(columns={'density_2019':'density'})
temp_2019_df = temp_2019_df.dropna(subset=['density'])
new_df = new_df.append(temp_2019_df)

In [157]:
new_df = new_df.dropna(subset=['Number_of_Users','Total_Payment'])

In [158]:
new_df['Payment_per_User'] = new_df['Total_Payment']/new_df['Number_of_Users']
new_df['ln_Payment_per_User'] = np.log(1+new_df['Payment_per_User'])
new_df['ln_Number_of_Users'] = np.log(1+new_df['Number_of_Users'])
new_df['ln_Number_of_Providers'] = np.log(1+new_df['Number_of_Providers'])
new_df['ln_Number_of_FFS_Beneficiaries'] = np.log(1+new_df['Number_of_Fee-for-Service_Beneficiaries'])
new_df['ln_Total_Payment'] = np.log(1+new_df['Total_Payment'])

In [159]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6519 entries, 4 to 9485
Data columns (total 14 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6519 non-null   object 
 1   state_county                                  5591 non-null   object 
 2   Number_of_Users                               6519 non-null   float64
 3   Percentage_of_Users_out_of_FFS_Beneficiaries  6519 non-null   float64
 4   Number_of_Fee-for-Service_Beneficiaries       6519 non-null   float64
 5   Number_of_Providers                           6519 non-null   float64
 6   Total_Payment                                 6519 non-null   float64
 7   density                                       6519 non-null   float64
 8   Payment_per_User                              6519 non-null   float64
 9   ln_Payment_per_User                           6519 non-null   f

In [160]:
new_df.describe()

,Number_of_Users,Percentage_of_Users_out_of_FFS_Beneficiaries,Number_of_Fee-for-Service_Beneficiaries,Number_of_Providers,Total_Payment,density,Payment_per_User,ln_Payment_per_User,ln_Number_of_Users,ln_Number_of_Providers,ln_Number_of_FFS_Beneficiaries,ln_Total_Payment
count,6519.000000,6519.000000,6519.000000,6519.000000,6.519000e+03,6519.000000,6519.000000,6519.000000,6519.000000,6519.000000,6519.000000,6519.000000
mean,62.374291,1.169871,15863.750422,1.560209,1.173628e+04,322.359843,191.774976,5.108269,3.765114,0.698195,8.772342,8.834915
std,72.093604,1.268147,36736.726221,2.210540,2.131591e+04,2112.278750,131.057031,0.521859,0.830591,0.656531,1.232928,1.003602
min,11.000000,0.010000,237.000000,0.000000,4.893700e+02,0.230884,30.315882,3.444125,2.484907,0.000000,5.472271,6.195160
25%,21.000000,0.310000,2800.000000,0.000000,3.276875e+03,22.528843,115.811061,4.760558,3.091042,0.000000,7.937732,8.094951
50%,39.000000,0.770000,5678.000000,1.000000,6.607400e+03,51.189055,153.529545,5.040385,3.688879,0.693147,8.644530,8.796097
75%,75.000000,1.550000,13781.000000,2.000000,1.402168e+04,140.430149,218.926987,5.393296,4.330733,1.098612,9.531118,9.548432
max,1251.000000,10.660000,815379.000000,26.000000,1.194943e+06,71427.858081,1996.949655,7.599877,7.132498,3.295837,13.611410,13.993610


In [161]:
new_df.corr()

,Number_of_Users,Percentage_of_Users_out_of_FFS_Beneficiaries,Number_of_Fee-for-Service_Beneficiaries,Number_of_Providers,Total_Payment,density,Payment_per_User,ln_Payment_per_User,ln_Number_of_Users,ln_Number_of_Providers,ln_Number_of_FFS_Beneficiaries,ln_Total_Payment
Number_of_Users,1.000000,0.231219,0.356421,0.826328,0.696658,0.171380,-0.023875,0.003230,0.841679,0.726783,0.356340,0.712107
Percentage_of_Users_out_of_FFS_Beneficiaries,0.231219,1.000000,-0.264560,0.262579,0.148147,-0.105736,0.026594,0.054048,0.315608,0.315603,-0.571908,0.296334
Number_of_Fee-for-Service_Beneficiaries,0.356421,-0.264560,1.000000,0.220568,0.269735,0.338044,0.026864,0.044304,0.242902,0.119552,0.659965,0.227869
Number_of_Providers,0.826328,0.262579,0.220568,1.000000,0.580881,0.189467,0.075886,0.091762,0.752838,0.893868,0.236985,0.683918
Total_Payment,0.696658,0.148147,0.269735,0.580881,1.000000,0.080941,0.313477,0.302244,0.534988,0.497474,0.227059,0.609349
density,0.171380,-0.105736,0.338044,0.189467,0.080941,1.000000,0.013246,0.019710,0.084407,0.067963,0.263446,0.081117
Payment_per_User,-0.023875,0.026594,0.026864,0.075886,0.313477,0.013246,1.000000,0.916229,-0.038637,0.076731,-0.038775,0.446171
ln_Payment_per_User,0.003230,0.054048,0.044304,0.091762,0.302244,0.019710,0.916229,1.000000,0.007222,0.090220,-0.022420,0.529332
ln_Number_of_Users,0.841679,0.315608,0.242902,0.752838,0.534988,0.084407,-0.038637,0.007222,1.000000,0.823881,0.358742,0.852166
ln_Number_of_Providers,0.726783,0.315603,0.119552,0.893868,0.497474,0.067963,0.076731,0.090220,0.823881,1.000000,0.188338,0.745317


Density 가 높을수록 ln_Number of User 도 조금 많아진다 --> Percentage of Users / FFS Beneficiaries 는 낮다

즉, Density 가 낮은 지역일수록 User Percentage 가 높다

Density 가 높을수록 Total Payment 가 조금 많다 --> Payment per User 도 조금 높다

즉, Density 가 낮은 지역일수록 Per User Payment 가 높다

Density 가 높을수록 Number of User, Number of FFS Beneficiaries 가 높아지고 Provider 도 많아진다 (당연)

### 2. Hospital Density

In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9523 entries, 0 to 9522
Data columns (total 73 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   id                                                      9523 non-null   object 
 1   POPESTIMATE2017                                         9477 non-null   float64
 2   POPESTIMATE2018                                         9477 non-null   float64
 3   POPESTIMATE2019                                         9477 non-null   float64
 4   total_land_water                                        9477 non-null   float64
 5   land_area                                               9477 non-null   float64
 6   water_area                                              9477 non-null   float64
 7   code                                                    9433 non-null   float64
 8   nchs_rurality                         

In [163]:
# Filter columns
temp_df = df[['id', 'state_county', 'year', 
              'Number_of_Users', 'Percentage_of_Users_out_of_FFS_Beneficiaries',
              'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
              'Total_Payment','hospital_density']]

In [165]:
# Exclude Alaska, Hawaii, and Maryland
temp_df['State_FIPS'] = [str(i)[:2] for i in temp_df['id']]
temp_df = temp_df[~temp_df['State_FIPS'].isin(['02','15','24'])]

In [166]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9342 entries, 0 to 9522
Data columns (total 10 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            9342 non-null   object 
 1   state_county                                  7296 non-null   object 
 2   year                                          9286 non-null   float64
 3   Number_of_Users                               6520 non-null   float64
 4   Percentage_of_Users_out_of_FFS_Beneficiaries  6520 non-null   float64
 5   Number_of_Fee-for-Service_Beneficiaries       6520 non-null   float64
 6   Number_of_Providers                           6520 non-null   float64
 7   Total_Payment                                 6520 non-null   float64
 8   hospital_density                              7284 non-null   float64
 9   State_FIPS                                    9342 non-null   o

In [172]:
new_df = pd.DataFrame()

In [173]:
temp_2017_df = temp_df[temp_df['year']==2017][['id', 'state_county', 'Number_of_Users', 
                                'Percentage_of_Users_out_of_FFS_Beneficiaries',
                                'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
                                'Total_Payment','hospital_density']]
temp_2017_df = temp_2017_df.dropna(subset=['hospital_density'])
new_df = new_df.append(temp_2017_df)

In [174]:
temp_2018_df = temp_df[temp_df['year']==2018][['id', 'state_county', 'Number_of_Users', 
                                'Percentage_of_Users_out_of_FFS_Beneficiaries',
                                'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
                                'Total_Payment','hospital_density']]
temp_2018_df = temp_2018_df.dropna(subset=['hospital_density'])
new_df = new_df.append(temp_2018_df)

In [175]:
temp_2019_df = temp_df[temp_df['year']==2019][['id', 'state_county', 'Number_of_Users', 
                                'Percentage_of_Users_out_of_FFS_Beneficiaries',
                                'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
                                'Total_Payment','hospital_density']]
temp_2019_df = temp_2019_df.dropna(subset=['hospital_density'])
new_df = new_df.append(temp_2019_df)

In [177]:
new_df = new_df.dropna(subset=['Number_of_Users','Total_Payment'])

In [178]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5591 entries, 4 to 9485
Data columns (total 8 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5591 non-null   object 
 1   state_county                                  5591 non-null   object 
 2   Number_of_Users                               5591 non-null   float64
 3   Percentage_of_Users_out_of_FFS_Beneficiaries  5591 non-null   float64
 4   Number_of_Fee-for-Service_Beneficiaries       5591 non-null   float64
 5   Number_of_Providers                           5591 non-null   float64
 6   Total_Payment                                 5591 non-null   float64
 7   hospital_density                              5591 non-null   float64
dtypes: float64(6), object(2)
memory usage: 393.1+ KB


In [179]:
new_df['Payment_per_User'] = new_df['Total_Payment']/new_df['Number_of_Users']
new_df['ln_Payment_per_User'] = np.log(1+new_df['Payment_per_User'])
new_df['ln_Number_of_Users'] = np.log(1+new_df['Number_of_Users'])
new_df['ln_Number_of_Providers'] = np.log(1+new_df['Number_of_Providers'])
new_df['ln_Number_of_FFS_Beneficiaries'] = np.log(1+new_df['Number_of_Fee-for-Service_Beneficiaries'])
new_df['ln_Total_Payment'] = np.log(1+new_df['Total_Payment'])

In [181]:
new_df.corr()

,Number_of_Users,Percentage_of_Users_out_of_FFS_Beneficiaries,Number_of_Fee-for-Service_Beneficiaries,Number_of_Providers,Total_Payment,hospital_density,Payment_per_User,ln_Payment_per_User,ln_Number_of_Users,ln_Number_of_Providers,ln_Number_of_FFS_Beneficiaries,ln_Total_Payment
Number_of_Users,1.000000,0.253270,0.345042,0.826466,0.693539,0.096491,-0.022841,0.001509,0.842356,0.730753,0.328621,0.718354
Percentage_of_Users_out_of_FFS_Beneficiaries,0.253270,1.000000,-0.272819,0.281308,0.158498,-0.092066,0.019175,0.051929,0.347401,0.340510,-0.575245,0.324476
Number_of_Fee-for-Service_Beneficiaries,0.345042,-0.272819,1.000000,0.211312,0.262048,0.274886,0.034141,0.049054,0.227516,0.107267,0.671313,0.218641
Number_of_Providers,0.826466,0.281308,0.211312,1.000000,0.573107,0.128576,0.067218,0.082955,0.752810,0.893073,0.211424,0.684441
Total_Payment,0.693539,0.158498,0.262048,0.573107,1.000000,0.057425,0.317593,0.300180,0.530546,0.491442,0.210162,0.606159
hospital_density,0.096491,-0.092066,0.274886,0.128576,0.057425,1.000000,0.054126,0.052567,0.042863,0.039184,0.214231,0.063202
Payment_per_User,-0.022841,0.019175,0.034141,0.067218,0.317593,0.054126,1.000000,0.917988,-0.041325,0.061960,-0.033770,0.436906
ln_Payment_per_User,0.001509,0.051929,0.049054,0.082955,0.300180,0.052567,0.917988,1.000000,0.002034,0.076243,-0.023078,0.516666
ln_Number_of_Users,0.842356,0.347401,0.227516,0.752810,0.530546,0.042863,-0.041325,0.002034,1.000000,0.826979,0.311410,0.857189
ln_Number_of_Providers,0.730753,0.340510,0.107267,0.893073,0.491442,0.039184,0.061960,0.076243,0.826979,1.000000,0.152108,0.746364


Hospital density 가 높을수록 ln_Number of User 도 조금 많아진다 --> Percentage of Users / FFS Beneficiaries 는 낮다

즉, Hospital density 가 낮은 지역일수록 User Percentage 가 높다

Hospital density 가 높을수록 Total Payment 가 조금 많다 --> Payment per User 도 조금 높다

즉, Hospital density 가 낮은 지역일수록 Per User Payment 가 높다

Hospital density 가 높을수록 Number of User, Number of FFS Beneficiaries 가 높아지고 Provider 도 많아진다 (당연)